In [47]:
import pandas as pd
import numpy as np
import pickle
from pandas import ExcelWriter
import ffn
%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

#pull in data from AlphaVantage
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='NXY0VT9AHBRYGKKC',output_format='pandas')



In [48]:
active_stocks = pd.read_csv('active_names.csv', index_col = 0).dropna()

In [49]:
def historical_data(ticker, outsize = "full"):
    #outsize can be compact and full. 
    #compact returns only the latest 100 data points; full 
    #returns the full-length time series of up to 20 years of historical data. The "compact" option is 
    #recommended if you would like to reduce the data size of each API call
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=NXY0VT9AHBRYGKKC&datatype=csv&outputsize={1}'.format(ticker, outsize)
    df = pd.read_csv(alphavantage_link)
    df.index = pd.to_datetime(df.timestamp)
    return df

In [60]:
Main_DF = historical_data("SPY")
Main_DF = Main_DF[['timestamp','close']]
Main_DF.columns = ["timestamp","SPY"]
Main_DF = Main_DF.sort_index()

In [51]:
MOMO = active_stocks.index[0:200]

In [52]:


for ticker in MOMO:
    temp_df = historical_data(ticker)
    temp_df.index = pd.to_datetime(temp_df.timestamp) #set the index as the timestamp
    temp_df = temp_df[['close']]
    temp_df.columns = [ticker]
    Main_DF = Main_DF.join(temp_df, how = "left")

Main_DF.dropna(inplace = True)
Main_DF = Main_DF.sort_index()
del Main_DF['timestamp']

In [54]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(Main_DF, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [55]:
with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [59]:
Main_DF

,timestamp,SPY
timestamp,,
2000-01-03,2000-01-03,145.4375
2000-01-04,2000-01-04,139.7500
2000-01-05,2000-01-05,140.0000
2000-01-06,2000-01-06,137.7500
2000-01-07,2000-01-07,145.7500
2000-01-10,2000-01-10,146.2500
2000-01-11,2000-01-11,144.5000
2000-01-12,2000-01-12,143.0625
2000-01-13,2000-01-13,145.0000
